In [8]:
#rede neural baseada em convolução
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import pickle as pck
import pandas as pd

In [6]:
#Carregando a representação
x = pck.load(open("datasets/representacao_tfidf_350.bin", "rb"))
x = x.toarray()
#Carregando o dataset
df = pd.read_csv("datasets/imdb.csv")
df = df[:350]

In [9]:

#representação binária dos rótulos
num_classes = 2
from sklearn.preprocessing import LabelEncoder
import numpy as np

y = np.array(df['sentiment'])

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
y = keras.utils.to_categorical(y, num_classes)


In [14]:
#padding sentenças (caso elas tenham tamanhos diferentes)
input_length = x.shape[1]
X = sequence.pad_sequences(x, maxlen=input_length)

In [18]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = .1)
num_classes = 2


In [19]:
# montando a rede CNN
model = Sequential()
model.add(Embedding(input_length, 32, input_length=input_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(num_classes, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 9965, 32)          318880    
                                                                 
 conv1d_2 (Conv1D)           (None, 9965, 32)          3104      
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 4982, 32)         0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 159424)            0         
                                                                 
 dense_4 (Dense)             (None, 250)               39856250  
                                                                 
 dense_5 (Dense)             (None, 2)                 502       
                                                      

In [ ]:
# Treinando o modelo
model.fit(x_train, y_train, validation_data=(x_val, y_val), 
          epochs=20, batch_size=128, verbose=1)
scores = model.evaluate(x_val, y_val, verbose=1)
print(scores)


Epoch 1/20
3/3 [==============================] - 4s 1s/step - loss: 0.6901 - accuracy: 0.5397 - val_loss: 0.7052 - val_accuracy: 0.4571
Epoch 2/20
3/3 [==============================] - 5s 1s/step - loss: 0.6902 - accuracy: 0.5397 - val_loss: 0.7044 - val_accuracy: 0.4571
Epoch 3/20
3/3 [==============================] - 6s 2s/step - loss: 0.6900 - accuracy: 0.5397 - val_loss: 0.7040 - val_accuracy: 0.4571
Epoch 4/20
2/3 [===================>..........] - ETA: 2s - loss: 0.6888 - accuracy: 0.5469